# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [113]:
NUM_VIDEOS = 200
GRID_SIZE = 2
BINS = [180, 256]
HIST_FRAME_SKIP = 20
REFRESH = False

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [114]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 200 - Grid 2 - Bins [180, 256] - Skip 20
Loading / processing dataset...
Done processing!

In [115]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = generate_test_segments(training_set, n=100, duration=20)

In [116]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:    {:d}".format( len(training_set)) )
print("Num. segments:  {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:       {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:     {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms: {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("\nTEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:    200
Num. segments:  30070
Duration:       97,332.7 s
Num frames:     2640076
Num histograms: 146284

TEST SET:
Size: 100


# Small manual test

In [134]:
pr = False
for i in range(3,4):
#     x = random.choice(range(len(test_set[i])))
    found = search.findFrame(test_set[i][0], training_set, cv2.HISTCMP_CHISQR, 2, prints= pr, warnings=pr)
    print('Found {} - Expected {}'.format(found, labels[i]))

Found ('00001.mp4', 0) - Expected ('00052.mp4', 3683, 4162)


## Run model on test set

In [135]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
# for method in [cv2.HISTCMP_BHATTACHARYYA]:
    %timeit -n 1 search.findFrame_old(test_set[0][0], training_set, method, warnings = False)
    %timeit -n 1 search.findFrame(test_set[0][0], training_set, method, warnings = False)
    print()

# for ch in [[0], [1], [0, 1]]:
#     print('{}'.format(ch))
#     %timeit -n 10 search.findFrame(test_set[0], training_set, cv2.HISTCMP_CORREL, channels=ch)

1 loop, best of 5: 970 ms per loop
1 loop, best of 5: 899 ms per loop

1 loop, best of 5: 1.06 s per loop
1 loop, best of 5: 897 ms per loop

1 loop, best of 5: 966 ms per loop
1 loop, best of 5: 816 ms per loop

1 loop, best of 5: 1.01 s per loop
1 loop, best of 5: 858 ms per loop

1 loop, best of 5: 1.24 s per loop
1 loop, best of 5: 1.02 s per loop

1 loop, best of 5: 1.07 s per loop
1 loop, best of 5: 952 ms per loop



In [127]:
results = []

for i, histogram in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(histogram)), end='', flush=True)
    
    results.append(search.findFrame(histogram[0], training_set, cv2.HISTCMP_CHISQR_ALT, 2, warnings = False))

Searching segment 100/100

## Evaluate performance

In [129]:
movie_results, start_frame_dist = evaluate(results, labels)

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS\n")

printParams()
print("\nCorrect video:                   {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment:                 {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to start frame: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/25))

TEST RESULTS

Num. Vid 200 - Grid 2 - Bins [180, 256] - Skip 20

Correct video:                   94 / 100 (94.0%)
Inside fragment:                 89 / 94 (94.7%)
Average distance to start frame: 93 +/- 102 frames (approx. 3.7 sec)
